# Introduction

In this tutorial we will use SQLite - this is a simple, file or memory based database. It is easy to use anywhere, as you just need the python module - no complicated installs or setup.

Here's the documentation: https://docs.python.org/3/library/sqlite3.html

Another introduction: https://www.pythoncentral.io/introduction-to-sqlite-in-python/ 

To get started, simply import the module:

In [4]:
import sqlite3

Now, we need to connect to a database. We will use a file - you could also use `":memory:"` to have a temporary database in memory. We also need a "cursor" - this points to the database, and lets us execute queries.

In [5]:
# Creates or opens a file called mydb with a SQLite3 DB
conn = sqlite3.connect('example_db')
# Get a cursor
cursor = conn.cursor()

# Creating Tables

The first thing to do is create a 'table' - tables have a name, and a set of columns. Each column has a particular type, and the database will complain if you try to put the wrong thing in it.

We'll make a table of students, with basic information in:
* an ID number (sid)
* a full name
* a Programme
* a year of study

Every time we have done something with the database, we have to commit the data.

In [6]:
cursor.execute('''
    CREATE TABLE students(id INTEGER PRIMARY KEY, name TEXT,
                       programme TEXT, year INTEGER)
''')
conn.commit()

OperationalError: table students already exists

In [7]:

# Run a SELECT query - more on this later
cursor.execute('''
    SELECT * from students
''')
# Get all of the results of the query, and print them
cursor.fetchall()

[(1, 'Kingsely', 'Music', 3),
 (2, 'JP', 'Geography', 3),
 (3, 'Jan', 'Medicine', 3),
 (4, 'Howard', 'Geography', 4)]

# Inserting Data

The previous command gave us an empty array, as we have not put anything into the table. We can put things in using `INSERT` statements. With `INSERT` queries, and most others, we use "placeholders" in the query - this is a questionmark (?) that will be replaced by a value that we pass in. This helps to avoid errors, and increases security.

In [8]:
# We use INSERT to put data into the table.
cursor.execute('''
    INSERT INTO students (id, name, programme, year) VALUES (?,?,?,?)
''', (1,"Kingsely","Geography", 3))
# Get all of the results of the query, and print them
conn.commit()

IntegrityError: UNIQUE constraint failed: students.id

In [9]:
cursor.execute('''
    SELECT * from students
''')
# Get all of the results of the query, and print them
cursor.fetchall()

[(1, 'Kingsely', 'Music', 3),
 (2, 'JP', 'Geography', 3),
 (3, 'Jan', 'Medicine', 3),
 (4, 'Howard', 'Geography', 4)]

If we try the same query again, it will fail, because we have said that the ID number has to be unique, so we can't add another student with ID 1. This is very useful - you know that if a column has a uniqueness constraint, all of the values will be unique:

In [10]:
# We use INSERT to put data into the table.
cursor.execute('''
    INSERT INTO students (id, name, programme, year) VALUES (?,?,?,?)
''', (1,"Kingsley","Geography", 3))
# Get all of the results of the query, and print them
conn.commit()

IntegrityError: UNIQUE constraint failed: students.id

It can be annoying adding rows one by one, so we can add multiple rows at once:

In [11]:
data = [
    (2, "JP","Geography", 3),
    (3, "Jan", "Medicine", 3)
]

cursor.executemany('''
    INSERT INTO students (id, name, programme, year) VALUES (?,?,?,?)
''', data)
conn.commit()


# Now check that they are in there
cursor.execute('''
    SELECT * from students
''')
cursor.fetchall()

IntegrityError: UNIQUE constraint failed: students.id

In [12]:
# It's also possible to use a dictionary, so that order doesn't matter.
# The keys in the dictionary should match what goes in the VALUES list, with a colon before each name. This
# may make it easier to use data from elsewhere.

data = { "student_id": 4, "year_of_study":4, "fullname":"Howard", "subject" : "Geography"}

cursor.execute('''
    INSERT INTO students (id, name, programme, year) VALUES (:student_id,:fullname,:subject,:year_of_study)
''', data)

conn.commit()


IntegrityError: UNIQUE constraint failed: students.id

# Retrieving data with SELECT

Now that we have data in thge database, we can use SELECT queries to retrieve it. So far, we have used SELECT with no options to get everything, but in practice we might want to:
    * only get certain rows
    * only get certain columns
    
Most `SELECT` queies have a `WHERE` clause, that defines the rows that you want. This means that a general `SELECT` looks like this:
    
`SELECT <cols> FROM <tablename> WHERE <constraints>`

Constraints can take many forms - the authorative document for SQLite is here, but it is hard reading: https://www.sqlite.org/lang_select.html. Common constraints are:
* `=` for equality (e.g. age = 45). Works with strings, but they need to be in quotes, e.g. `name = 'Jemima'`
* `>` or `<` for numbers
* `LIKE` does partial string matching, with `%` standing in for "anything", so `city LIKE 'Lon%'` would match London, Long Beach and Longjing
* `NOT` selects all the rows where the condition does not match
* `AND`, OR can be used to combine conditions

A `*` for cols means get all of the columns, but you can also use comma separated column names to get less of the data.       

In [13]:
# Get just students in year 3
cursor.execute('''
    SELECT * from students WHERE year = 3
''')
cursor.fetchall()

[(1, 'Kingsely', 'Music', 3),
 (2, 'JP', 'Geography', 3),
 (3, 'Jan', 'Medicine', 3)]

In [14]:
# Get just the name and ID columns
cursor.execute('''
    SELECT name, id from students 
''')
cursor.fetchall()

[('Kingsely', 1), ('JP', 2), ('Jan', 3), ('Howard', 4)]

## Exercises:

In [15]:
# Get all of the Geography students
cursor.execute('''
SELECT * from students WHERE programme='Geography'
''')

cursor.fetchall()

[(2, 'JP', 'Geography', 3), (4, 'Howard', 'Geography', 4)]

In [16]:
# Get all of the students who are *NOT* in year 3
cursor.execute('''
SELECT * from students WHERE year IS NOT 3
''')

cursor.fetchall()

[(4, 'Howard', 'Geography', 4)]

In [17]:
# Get all of the students whose name starts with "J"
cursor.execute('''
SELECT * FROM students WHERE name LIKE 'J%'
''')

cursor.fetchall()

[(2, 'JP', 'Geography', 3), (3, 'Jan', 'Medicine', 3)]

In [18]:
# Get all of the students who are in year 3 *AND* studying Geography. Do the same with OR
cursor.execute('''
SELECT * from students WHERE year=3 AND programme='Geography'
''')

cursor.fetchall()

[(2, 'JP', 'Geography', 3)]

In [19]:
# Read all the data from the csv file `extra_students.csv` and add this to your database

# Start by opening the file with a CSV reader
import pandas as pd

data = pd.read_csv('extra_students.csv', header=None)

# print(data.head())
# Now go through each line in the file
for line in data.values:
    print(line)
# Construct an insert statement that will add that row to the database. Note that you will have to turn the ID and Year fields into Integers
    cursor.execute('''
    INSERT INTO students(id, name, programme, year) VALUES (?,?,?,?)
     ''', (int(line[0]), line[1], line[2], int(line[3]))
    )
# When you have added the whole file, make sure to commit it.
conn.commit()

[5 'Vod' 'English' 3]
[6 'Nevile' 'Maths' 2]
[7 'Chris' 'Maths' 2]


In [20]:
# Now select all of the students to make sure you have a full set of 7.
cursor.execute('''
    SELECT * from students
''')
cursor.fetchall()

[(1, 'Kingsely', 'Music', 3),
 (2, 'JP', 'Geography', 3),
 (3, 'Jan', 'Medicine', 3),
 (4, 'Howard', 'Geography', 4),
 (5, 'Vod', 'English', 3),
 (6, 'Nevile', 'Maths', 2),
 (7, 'Chris', 'Maths', 2)]

# Changing data

We saw that we cannot add another student with the same ID. However, we might want to change some of the information - perhaps we found that Kingsley is actually studing Music, or that English should be called English Literature. The `UPDATE` statement does this. It needs:
* a set of `column=value` pairs for the new data
* a set of rows to work on (using a `WHERE` clause).
So it looks like:
`UPDATE <tablename> SET <col1=val1,col2=val2,...> WHERE <constraints>`

In [21]:
# This example will change Kingsley's programme to Music, based on knowing his ID is 1
cursor.execute('''
    UPDATE students SET programme="Music" WHERE id = 1
''')
conn.commit()
cursor.execute('''
    SELECT * from students
''')
cursor.fetchall()

[(1, 'Kingsely', 'Music', 3),
 (2, 'JP', 'Geography', 3),
 (3, 'Jan', 'Medicine', 3),
 (4, 'Howard', 'Geography', 4),
 (5, 'Vod', 'English', 3),
 (6, 'Nevile', 'Maths', 2),
 (7, 'Chris', 'Maths', 2)]

In [22]:
# Now, change all of the students who are doing English so that they are doing "English Literature"

cursor.execute('''
    UPDATE students SET programme="English Literature" WHERE programme="English"
''')
conn.commit()
cursor.execute('''
    SELECT * from students
''')
cursor.fetchall()

[(1, 'Kingsely', 'Music', 3),
 (2, 'JP', 'Geography', 3),
 (3, 'Jan', 'Medicine', 3),
 (4, 'Howard', 'Geography', 4),
 (5, 'Vod', 'English Literature', 3),
 (6, 'Nevile', 'Maths', 2),
 (7, 'Chris', 'Maths', 2)]

# Deleting Data

Deleting uses the `DELETE` command, with a `WHERE` clause, to delete all of the rows that match the condition, e.g.

In [23]:
# This command will delete all the students whose ID is greater than 4
cursor.execute('''
    DELETE FROM students WHERE id > 4
''')
conn.commit()

# Joining Data

Data can be joined together in different ways (Refer back to the end of Lecture 2 for pictures of the operations).

In SQL we typically use `JOIN` to bring two separate datasets together. This is a very quick intro. Some more examples with SQLite are in: https://www.tutorialspoint.com/sqlite/sqlite_using_joins.htm 

To demonstrate this, we'll create a table of extra information about students:

In [24]:
data = [(1, 'Swimming'), (2, 'Fencing'), (5, 'Swimming'), (6, 'Trampolining'), (7, 'Tennis')]

cursor.execute('''
    CREATE TABLE hobbies(student_id INTEGER, hobby TEXT)
''')

cursor.executemany('''
    INSERT INTO hobbies (student_id, hobby) VALUES (?,?)
''', data)
conn.commit()

OperationalError: table hobbies already exists

Now that we have the table, we can use SELECT with JOIN to get joined up data. The first kind of join is an `INNER JOIN` - this will give a row in the output for every row which exists in *both* tables:

In [25]:
cursor.execute('''
    SELECT * from students JOIN hobbies ON students.id == hobbies.student_id
''')
cursor.fetchall()

[(1, 'Kingsely', 'Music', 3, 1, 'Swimming'),
 (2, 'JP', 'Geography', 3, 2, 'Fencing')]

As you can see, this has only given us a row for students that have a hobby. Often, we would be interested in *all* students, even if they don't have a hobby. We can do this using an `OUTER JOIN`. There are three kinds of outer join:
* `LEFT OUTER JOIN` gives a row for each row in the left hand table (students in this case) whether or not there is a row in the right hand table (hobbies).
* `RIGHT OUTER JOIN` gives a row for each row in the right hand table, whether or not there is a corresponding row in the left hand table
* `FULL OUTER JOIN` gives a row for each row in *either* table.

SQLite only support `LEFT OUTER JOIN`, but other databases will let you do more.

In [26]:
cursor.execute('''
    SELECT * from students LEFT OUTER JOIN hobbies ON students.id == hobbies.student_id
''')
cursor.fetchall()

[(1, 'Kingsely', 'Music', 3, 1, 'Swimming'),
 (2, 'JP', 'Geography', 3, 2, 'Fencing'),
 (3, 'Jan', 'Medicine', 3, None, None),
 (4, 'Howard', 'Geography', 4, None, None)]

## Excercise
* Create a new database table with some other information about the students in
* Join that onto the original info, as above.

In [27]:
data = [(1, 'Student Union'), (2, 'Broadcast Station')]
# Create table
cursor.execute('''
     CREATE TABLE responsibilities(student_id INTEGER, responsibility TEXT)
''')
# Fill it with students
cursor.executemany('''
    INSERT INTO responsibilities (student_id, responsibility) VALUES (?,?)
''', data)

conn.commit()
cursor.execute('''
SELECT * FROM responsibilities
''')
cursor.fetchall()

OperationalError: table responsibilities already exists

In [28]:
# Now construct a join
cursor.execute('''
SELECT * FROM students JOIN responsibilities ON students.id=responsibilities.student_id
''')

cursor.fetchall()

[(1, 'Kingsely', 'Music', 3, 1, 'Student Union'),
 (2, 'JP', 'Geography', 3, 2, 'Broadcast Station')]

# Pandas and SQLite

Some of the operations on databases are a bit more fiddly than you are used to. The good news is that `pandas` can be used alongside SQLite, relating `DataFrames` to database tables.

Some resources:
* Documentation: https://pandas.pydata.org/pandas-docs/stable/io.html#sql-queries
* Tutorial mapping SQL to Pandas: https://medium.com/jbennetcodes/how-to-rewrite-your-sql-queries-in-pandas-and-more-149d341fc53e 

This means that you can use advanced Pandas features that make it easy to read in CSV files with the correct types, and then put the data in a database so it can be queried and used in the future. You can bring the results of complex JOINs into DataFrames and then work with them conveniently in-memory.

Firstly, it is easy to get the result of a `SELECT` query as a `DataFrame`:

In [29]:
import pandas as pd
conn.commit()
df = pd.read_sql_query("select * from students", con=conn)
df

# NOTE: this seems to sometimes return empty tables - it's hopefully OK on the notebook servers!

,id,name,programme,year
0,1,Kingsely,Music,3
1,2,JP,Geography,3
2,3,Jan,Medicine,3
3,4,Howard,Geography,4


Next, we can add dataframes as tables. This will make the dataframe persistent, and allow it to be used with other SQL select queries etc:

In [45]:
df = pd.DataFrame(data={"id":[5,6,7],"food":["Cheese", "Bread","Leuttice"]})
df

,id,food
0,5,Cheese
1,6,Bread
2,7,Leuttice


In [31]:
df.to_sql('food', conn,if_exists="replace") 
#The if_exists field can also be set to "append" to 
# add data to an existing table, or "fail" to stop if the table already exists
cursor.execute('''
    SELECT * from food
''')
cursor.fetchall()

[(0, 5, 'Cheese'), (1, 6, 'Bread'), (2, 7, 'Leuttice')]

## Exercise

Use Pandas' CSV reading function to read in the extra_students data to a DataFrame, and then create a table in the database with that data. Use a `SELECT` query to make sure you have the right data in there.

In [47]:
cursor.execute('''
    CREATE TABLE extraStudents(id INTEGER PRIMARY KEY, name TEXT, programme TEXT, year INTEGER)
''')

conn.commit()

OperationalError: table extraStudents already exists

In [60]:
data = pd.read_csv('extra_students.csv', header=None, names=["id","name","programme","year"])
data

,id,name,programme,year
0,5,Vod,English,3
1,6,Nevile,Maths,2
2,7,Chris,Maths,2


In [59]:
data.to_sql('my_extraStudents', conn, if_exists="replace")

cursor.execute('''
    SELECT * from my_extraStudents
''')

cursor.fetchall()

[(0, '5', 'Vod', 'English', 3),
 (1, '6', 'Nevile', 'Maths', 2),
 (2, 'aa7', 'Chris', 'Maths', 2)]

# Summing up

Through this notebook, you have seen:
* How to connect to a simple SQL database
* How to add, select, modify and delete data within the database
* How to use Pandas to interface more easily with the data

## Useful commands

There are some commands below that may be of use for setting and checking the database state.

In [61]:
# Completely delete the students table
cursor.execute('''
    DROP TABLE students
''')
conn.commit()

In [62]:
# Keep the structure of the table, but delete all of the rows in it
cursor.execute('''
    DELETE from students
''')
conn.commit()

OperationalError: no such table: students

In [ ]:
# Show all of the data in the students table
cursor.execute('''
    SELECT * from students
''')
cursor.fetchall()


In [63]:
# Show all of the data in the hobbies table
cursor.execute('''
    SELECT * from hobbies
''')
print( cursor.fetchall() )

[(1, 'Swimming'), (2, 'Fencing'), (5, 'Swimming'), (6, 'Trampolining'), (7, 'Tennis')]


In [64]:
# Setup the database connection again

# Creates or opens a file called mydb with a SQLite3 DB
conn = sqlite3.connect('example_db')
# Get a cursor
cursor = conn.cursor()

In [65]:
# Make sure all the data is commited to the database
conn.commit()

In [66]:
# Close the connection to the database
# It's good practice to do this when you have finished working with the data
conn.close()